In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [34]:
X = pd.read_csv('references/training_set_values.csv', index_col = 'id')
y = pd.read_csv('references/training_set_labels.csv', index_col = 'id')['status_group']

In [35]:
X.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [36]:
y

id
69572        functional
8776         functional
34310        functional
67743    non functional
19728        functional
              ...      
60739        functional
27263        functional
37057        functional
31282        functional
26348        functional
Name: status_group, Length: 59400, dtype: object

In [37]:
# columns to drop
'date_recorded', 'longitude', 'latitude', 'num_private', 

('date_recorded', 'longitude', 'latitude', 'num_private')

In [38]:
# columns to change
'construction_year'

'construction_year'

In [39]:
X_num = X.select_dtypes(exclude = ['object'])
X_train, X_test, y_train, y_test = train_test_split(X_num, y, random_state = 42, test_size = 0.25)
lg = LogisticRegression(max_iter = 1000)
lg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
cross_val_score(lg, X_train, y_train, cv = 5)

array([0.55566779, 0.55185185, 0.54848485, 0.54893378, 0.55252525])